In [ ]:
steps = 300
overlap = 100

results_iter = {}

optm = Adam(lr=1e-3)

epochs=50
batch_size=128

for steps in range(350, 800, 50):
    for overlap in range(25,175,25):
        
        test_train_x = []
        test_train_y = []
        
        for key in data:
            for drive in data[key]:
                x_windows, y_windows = split_sequences(data[key][drive].values, steps, key, overlap)
                test_train_x.extend(x_windows)
                test_train_y.extend(y_windows)
                
        if steps == 300:
            continue
        
        y = array(test_train_y)
        x = array(test_train_x)
        
        scaler = MinMaxScaler(feature_range=(0, 1))

        for data_x in x:
            scaler.partial_fit(data_x)

        scaled_x = []
        # Second pass
        for data_x in x:
            scaled_x.append(list(map(list, zip(*scaler.transform(data_x)))))
            
        x_train, x_test, y_train_numbers, y_test_numbers = train_test_split(array(scaled_x), array(y), test_size=0.33, random_state=42)
        print(steps)
        print(overlap)
        print(x_train.shape)
        
        model = generate_model()
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
        classes = np.unique(y_train_numbers)
        le = LabelEncoder()
        y_ind = le.fit_transform(y_train_numbers.ravel())
        recip_freq = len(y_train_numbers) / (len(le.classes_) * np.bincount(y_ind).astype(np.float64))
        class_weight = recip_freq[le.transform(classes)]
        
        y_train = to_categorical(y_train_numbers, len(np.unique(y_train_numbers)))
        y_test = to_categorical(y_test_numbers, len(np.unique(y_test_numbers)))

        model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                      class_weight=class_weight, verbose=2, validation_data=(x_test, y_test))
        
        y_pred = model.predict(x_test, batch_size=64, verbose=1)
        y_pred_bool = np.argmax(y_pred, axis=1)
        y_pred = to_categorical(y_pred_bool, len(np.unique(y_pred_bool)))
        print(classification_report(y_test, y_pred))
        
        results_iter[str(steps)+'_'+str(overlap)] = classification_report(y_test, y_pred)